# Targets per Model (5–60 min, step 5)

**Common setup**

- **Anchor time:** $t_0$ (first bar of the segment)  
- **Horizons:** $K = \{5, 10, 15, \ldots, 60\}$ minutes  
- **Features:** computed causally from the 120-bar lookback up to $t_0$  
- No sequences exceed 60 bars ⇒ no right-censoring; still use a horizon mask if $(t_0 + k)$ crosses a session boundary.

---

## 1. Price — Directional Accuracy

Label per horizon $k$:

$$
y^{(\text{dir})}_k =
\begin{cases}
1, & \text{if } \displaystyle\sum_{i=1}^{k} \text{ret}_1(t_0 + i) > 0, \\[6pt]
-1, & \text{otherwise.}
\end{cases}
$$


Shape: `(12,)` boolean vector for $k \in K$.

---

## 2. Price — Magnitude (optional)

Label per horizon $k$:

$$
y^{(\text{bp})}_k = \sum_{i=1}^{k} \text{ret}_1(t_0 + i)
$$

Expressed in % or basis points; **do not** z-score the label.  
Shape: `(12,)` float vector.

---

## 3. Continuation / Survival (segment persistence)

Let $T$ be the number of bars in the segment from $t_0$ to its end.  
Label per horizon $k$:

$$
y^{(\text{alive})}_k =
\begin{cases}
1, & \text{if } T \ge k \\
0, & \text{otherwise}
\end{cases}
$$

Shape: `(12,)` boolean vector.

---

## 4. Volatility Regime

Compute realized volatility over $(t_0, t_0 + k]$:

$$
\sigma_k = \sqrt{ \sum_{i=1}^{k} \text{ret}_1(t_0 + i)^2 }
$$

Define thresholds on the **train split** (e.g., high-vol quantile).  
Label per horizon $k$:

$$
y^{(\text{vol})}_k =
\begin{cases}
1, & \text{if } \sigma_k \ge \text{threshold}_{\text{high}} \\
-1, & \text{if } \sigma_k \ge \text{threshold}_{\text{low}} \\
0, & \text{otherwise}
\end{cases}
$$

Shape: `(12,)` boolean vector.

---

## 5. Liquidity Regime

Compute average future volume proxy over $(t_0, t_0 + k]$, e.g. mean of $\text{vol\_z\_60}$.  
Threshold on **train split**.  
Label per horizon $k$:

$$
y^{(\text{liq})}_k =
\begin{cases}
1, & \text{if } \text{liq}_k \ge \text{threshold}_{\text{high}} \\
0, & \text{otherwise}
\end{cases}
$$

Shape: `(12,)` boolean vector.

---

## 6. Drift (directional bias)

Compute average return over $(t_0, t_0 + k]$:

$$
\text{drift}_k = \frac{1}{k} \sum_{i=1}^{k} \text{ret}_1(t_0 + i)
$$

Thresholds on **train split** for high/low drift.  
Label per horizon $k$:

$$
y^{(\text{drift})}_k =
\begin{cases}
1, & \text{if } \text{drift}_k \ge \text{threshold}_{\text{pos}} \\
0, & \text{otherwise}
\end{cases}
$$

Shape: `(12,)` boolean vector.

---

## 7. Order-Flow Imbalance (OFI)

Compute OFI proxy (e.g., CMF–OBV composite) over $(t_0, t_0 + k]$.  
Quantile thresholds on **train split** to define buy/sell/neutral.  
Label per horizon $k$:

$$
y^{(\text{ofi})}_k =
\begin{cases}
1, & \text{if } \text{ofi}_k \ge \text{threshold}_{\text{buy}} \\
0, & \text{otherwise}
\end{cases}
$$

Shape: `(12,)` boolean vector (or integer vector if using $\{-1,0,+1\}$).

---

## Masks

Define a horizon mask $m_k$:

$$
m_k =
\begin{cases}
1, & \text{if the window } (t_0, t_0 + k] \text{ is complete} \\
0, & \text{otherwise}
\end{cases}
$$

Compute the total loss as:

$$
\mathcal{L} = \sum_{k} m_k \cdot \mathcal{L}_k
$$



#### So each model will have a target sequence of length 12 representing 12 5 minute intervals and there will be binary features indicating whether the condition used to select the original segment is valid or not, but each model aims to predict a target at 5-60 minute horizon intervals.